In [6]:
#Importacion de las librerias
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from datetime import datetime
import psycopg2
import pyodbc

In [62]:
#Creacion de los data frame de la base de datos chinook
engine = create_engine('sqlite:///chinook.db')
df_customers = pd.read_sql_query("""SELECT FirstName, LastName, Phone, Fax, Email FROM customers; """, con=engine.connect())
df_artists = pd.read_sql_query("""SELECT Name FROM artists;""", con=engine.connect())
df_invoice_items = pd.read_sql_query("""SELECT UnitPrice, Quantity FROM invoice_items;""", con=engine.connect())
df_location = pd.read_sql_query("""SELECT BillingAddress as Address, BillingCity as City, BillingState as State, BillingCountry as Country,BillingPostalCode as PostalCode from invoices;""", con=engine.connect())
df_employees = pd.read_sql_query("""SELECT FirstName, LastName, Title, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email From employees;""", con=engine.connect())
df_albums = pd.read_sql_query("""SELECT Title FROM albums;""", con=engine.connect()) 
df_tracks = pd.read_sql_query("""SELECT t.Name, mt.Name AS MediaType, g.Name AS Genre, t.Composer, t.Milliseconds, t.Bytes, t.UnitPrice 
       FROM tracks t
       INNER JOIN
       [genres] g ON t.GenreId = g.GenreId
       INNER JOIN
       media_types mt ON t.MediaTypeId = mt.MediaTypeId """, con=engine.connect()) 
df_fs=pd.read_sql_query("""SELECT invoice_items.InvoiceLineId as InvoicesID, customers.CustomerId AS CustomerID, employees.EmployeeId AS EmployeeID, dim_time.timeID AS TimeID,
invoices.InvoiceId as LocationID, tracks.TrackId AS TrackId, artists.ArtistId AS ArtistId, albums.AlbumId AS AlbumId, invoices.Total AS Total
FROM invoices INNER JOIN customers
ON invoices.CustomerId= customers.CustomerId
INNER JOIN invoice_items
ON invoice_items.InvoiceId= invoices.InvoiceId
INNER JOIN tracks
ON tracks.TrackId=  invoice_items.TrackId
INNER JOIN albums
ON tracks.AlbumId= albums.AlbumId
INNER JOIN artists 
ON albums.ArtistId= artists.ArtistId 
INNER JOIN Dim_Time
ON invoices.InvoiceDate= substr(dim_time.date,0,20)
INNER JOIN employees
ON employees.EmployeeId= customers.SupportRepId;""", con=engine.connect())

In [75]:
#Limpieza de los datos
df_location.fillna(method="bfill", inplace = True)
df_employees.fillna(method="bfill", inplace = True)
df_customers.fillna(method="ffill", inplace = True)
df_tracks.fillna(method="bfill", inplace = True)

In [42]:
#Creacion del data frame de fecha
fecha_I='2009-01-01'
fecha_F='2030-12-31'
df_date = pd.DataFrame({'date': pd.date_range(fecha_I, fecha_F)})
df_date['año'] = df_date.date.dt.year
df_date['mes'] = df_date.date.dt.month
df_date['dia'] = df_date.date.dt.day
df_date['nombre_dia'] = df_date.date.dt.day_name()
df_date['dia_semana'] = df_date.date.dt.dayofweek
df_date['semana'] = df_date.date.dt.weekofyear
df_date['trimestre'] = df_date.date.dt.quarter
df_date = dfdate[['date', 'año', 'mes', 'dia', 'nombre_dia', 'dia_semana', 'semana', 'trimestre']]
df_date.to_sql(name='Dim_Time', con=engine.connect(), if_exists='append',index=False)

C:\Users\pined\AppData\Local\Temp/ipykernel_9356/2191331294.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_date['semana'] = df_date.date.dt.weekofyear


In [76]:
#Conexion con SQL
conn = sqlalchemy.create_engine('mssql+pyodbc://localhost/DW_Sales_Music?driver=SQL+Server+Native+Client+11.0')

In [88]:
#Carga de los datos en SQL
df_albums.to_sql(name='Dim_Album', con=conn, if_exists='append',index=False)
df_artists.to_sql(name='Dim_Artist', con=conn, if_exists='append',index=False)
df_customers.to_sql(name='Dim_Customer', con=conn, if_exists='append',index=False)  
df_invoice_items.to_sql(name='Dim_Invoice_Item', con=conn, if_exists='append',index=False)
df_location.to_sql(name='Dim_Location', con=conn, if_exists='append',index=False)
df_date.to_sql(name='Dim_Time', con=conn, if_exists='append',index=False)
df_tracks.to_sql(name='Dim_Track', con=conn, if_exists='append',index=False)
df_employees.to_sql(name='Dim_Employee', con=conn, if_exists='append',index=False)
df_fs.to_sql(name='Fact_Sales', con=conn, if_exists='append',index=False)